### Touch

Имеем два набора данных. один набор размечан пользователями 28 примеров, второй набор размечан автоматически 3188.

Проверим, насколько автоматически полученные данные, хорошо описывают размечанные вручную.


In [2]:
import matplotlib.pyplot as plt
import pandas as pn
import numpy as np
%matplotlib inline  


In [4]:
file_name_user = 'data_sets/mphone_mark_users.csv'
file_name_auto = 'data_sets/mphone_mark_auto.csv'

#?pn.read_csv
columns = ["Gender","Fingerprint","TimeUpx","W","H","leng","leng_sum","speed","Force","ForceUp","Rx","Ry","varX","varY"]
print('columns: %d' % len(columns))
df_user = pn.read_csv(file_name_user, nrows=15000, names=columns, header=1)
df_auto = pn.read_csv(file_name_auto, nrows=15000, names=columns, header=1)

columns: 14


In [5]:
df_user.describe()

,TimeUpx,W,H,leng,leng_sum,speed,Force,ForceUp,Rx,Ry,varX,varY
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,2.700000e+01
mean,373.723473,350.370370,662.592593,177.994823,1216.167096,0.483212,0.452393,0.002481,12.715362,12.715362,2086.672672,1.793523e+05
std,302.411272,20.751549,75.541257,296.190706,1043.608911,0.349451,0.499913,0.007526,15.919060,15.919060,3643.059642,4.694765e+05
min,0.000000,320.000000,568.000000,24.745730,77.820310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,184.715000,320.000000,568.000000,50.585805,386.837215,0.193145,0.000000,0.000000,0.000000,0.000000,113.070085,7.912361e+02
50%,295.000000,360.000000,667.000000,108.321700,720.271370,0.434170,0.056330,0.000000,0.659180,0.659180,538.249140,4.420800e+03
75%,454.083335,360.000000,740.000000,147.380345,2233.540275,0.757295,1.000000,0.000000,28.601315,28.601315,2109.694450,1.430145e+05
max,1385.666670,375.000000,760.000000,1591.342210,3439.153620,1.155530,1.000000,0.031670,44.886670,44.886670,14559.183400,2.301841e+06


In [6]:
df_auto.describe()

,TimeUpx,W,H,leng,leng_sum,speed,Force,ForceUp,Rx,Ry,varX,varY
count,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000,3188.000000
mean,771.764652,368.568346,639.807897,138.054755,2264.775905,0.359535,0.803830,0.070930,2.168585,2.204334,1788.098974,3122.489911
std,1171.353036,77.293368,68.062270,73.612584,2623.534999,0.298511,0.365201,0.252262,3.192029,3.179487,6511.340500,4394.646649
min,0.000000,320.000000,320.000000,0.000000,0.000000,0.000000,0.003930,0.000000,0.000000,0.000000,0.000000,0.000000
25%,243.895837,360.000000,640.000000,83.697188,856.507152,0.205483,1.000000,0.000000,0.000000,0.000000,111.687115,562.264833
50%,460.525645,360.000000,640.000000,120.338900,1535.639780,0.318375,1.000000,0.000000,0.708340,0.793565,343.992190,1638.736115
75%,854.645243,360.000000,640.000000,174.479582,2638.827905,0.454695,1.000000,0.000000,3.291508,3.291508,1108.977437,3897.983607
max,20662.000000,1280.000000,1280.000000,584.874850,34518.969310,10.135830,1.091670,1.000000,22.575510,22.575510,198307.582230,51175.770670


In [7]:
from sklearn.preprocessing import MinMaxScaler

def map_target(cl):
    if(cl == 'f'):
        return 0
    return 1

X = df_auto[columns[2:]]
X_test =df_user[columns[2:]]
Y = df_auto['Gender'].map(map_target)
Y_test = df_user['Gender'].map(map_target) 

scaler = MinMaxScaler()
X_std = scaler.fit_transform(X)
X_test_std = scaler.transform(X_test)

#### RandomForestClassifier

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=20, max_depth=4, random_state=0)
clf.fit(X_std,Y)
print(clf.score(X_test_std, Y_test))

0.4074074074074074


In [9]:
print classification_report(clf.predict(X_test_std), Y_test)

             precision    recall  f1-score   support

          0       0.22      0.18      0.20        11
          1       0.50      0.56      0.53        16

avg / total       0.39      0.41      0.40        27



#### GradientBoostingClassifier

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(max_depth=4, random_state=42)
clf.fit(X_std,Y)
print(clf.score(X_test_std, Y_test))

0.7407407407407407


In [11]:
print classification_report(clf.predict(X_test_std), Y_test)

             precision    recall  f1-score   support

          0       0.78      0.58      0.67        12
          1       0.72      0.87      0.79        15

avg / total       0.75      0.74      0.73        27



####  NN keras

In [12]:
from keras import Sequential
from keras.layers import Dense

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
classifier = Sequential() 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=len(X_std[0]))) 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [22]:
classifier.fit(X_std,Y, batch_size=10, epochs=10,validation_data=(X_test_std, Y_test))

Train on 3188 samples, validate on 27 samples
Epoch 1/10
3188/3188 [==============================] - 1s 187us/step - loss: 0.3917 - acc: 0.8218 - val_loss: 1.5741 - val_acc: 0.5926
Epoch 2/10
3188/3188 [==============================] - 1s 181us/step - loss: 0.3769 - acc: 0.8269 - val_loss: 1.7234 - val_acc: 0.5926
Epoch 3/10
3188/3188 [==============================] - 1s 175us/step - loss: 0.3710 - acc: 0.8312 - val_loss: 1.9062 - val_acc: 0.5926
Epoch 4/10
3188/3188 [==============================] - 1s 177us/step - loss: 0.3697 - acc: 0.8312 - val_loss: 1.9527 - val_acc: 0.6296
Epoch 5/10
3188/3188 [==============================] - 1s 184us/step - loss: 0.3660 - acc: 0.8366 - val_loss: 1.9923 - val_acc: 0.5926
Epoch 6/10
3188/3188 [==============================] - 1s 175us/step - loss: 0.3646 - acc: 0.8366 - val_loss: 2.2228 - val_acc: 0.6296
Epoch 7/10
3188/3188 [==============================] - 1s 178us/step - loss: 0.3674 - acc: 0.8359 - val_loss: 2.0932 - val_acc: 0.6667
Ep

In [23]:
classifier.fit(X_test_std,Y_test, batch_size=10, epochs=10,validation_data=(X_test_std, Y_test))

Train on 27 samples, validate on 27 samples
Epoch 1/10
27/27 [==============================] - 0s 533us/step - loss: 2.3305 - acc: 0.6296 - val_loss: 1.5627 - val_acc: 0.5926
Epoch 2/10
27/27 [==============================] - 0s 500us/step - loss: 1.8308 - acc: 0.5926 - val_loss: 1.5377 - val_acc: 0.6296
Epoch 3/10
27/27 [==============================] - 0s 605us/step - loss: 1.3983 - acc: 0.7037 - val_loss: 1.4320 - val_acc: 0.6667
Epoch 4/10
27/27 [==============================] - 0s 523us/step - loss: 1.3995 - acc: 0.7037 - val_loss: 1.2509 - val_acc: 0.7037
Epoch 5/10
27/27 [==============================] - 0s 538us/step - loss: 1.2483 - acc: 0.7407 - val_loss: 1.1513 - val_acc: 0.8148
Epoch 6/10
27/27 [==============================] - 0s 431us/step - loss: 1.1774 - acc: 0.8148 - val_loss: 1.0741 - val_acc: 0.8148
Epoch 7/10
27/27 [==============================] - 0s 417us/step - loss: 1.0497 - acc: 0.8148 - val_loss: 0.9978 - val_acc: 0.8519
Epoch 8/10
27/27 [==============

### Split train data

In [24]:
from sklearn.model_selection import train_test_split

X_train_std_, X_test_std_, y_train_std_, y_test_std_ = train_test_split(X_std, Y, test_size=0.33, random_state=42)
print(len(X_train_std_))

2135


### GradientBoostingClassifier

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(max_depth=4, random_state=42)
clf.fit(X_train_std_,y_train_std_)
print(clf.score(X_test_std_, y_test_std_))

0.8271604938271605


### NN Keras

In [ ]:
from keras import Sequential
from keras.layers import Dense

In [27]:
classifier = Sequential() 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=len(X_std[0]))) 
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal')) 
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [30]:
classifier.fit(X_train_std_,y_train_std_, batch_size=10, epochs=10,validation_data=(X_test_std_, y_test_std_))

Train on 2135 samples, validate on 1053 samples
Epoch 1/10
2135/2135 [==============================] - 0s 217us/step - loss: 0.4134 - acc: 0.8103 - val_loss: 0.4680 - val_acc: 0.7854
Epoch 2/10
2135/2135 [==============================] - 0s 216us/step - loss: 0.4183 - acc: 0.8056 - val_loss: 0.4740 - val_acc: 0.7683
Epoch 3/10
2135/2135 [==============================] - 0s 218us/step - loss: 0.4183 - acc: 0.8066 - val_loss: 0.4867 - val_acc: 0.7626
Epoch 4/10
2135/2135 [==============================] - 0s 215us/step - loss: 0.4044 - acc: 0.8192 - val_loss: 0.4530 - val_acc: 0.7968
Epoch 5/10
2135/2135 [==============================] - 0s 219us/step - loss: 0.4037 - acc: 0.8178 - val_loss: 0.4526 - val_acc: 0.7968
Epoch 6/10
2135/2135 [==============================] - 1s 238us/step - loss: 0.4001 - acc: 0.8197 - val_loss: 0.4497 - val_acc: 0.7901
Epoch 7/10
2135/2135 [==============================] - 0s 233us/step - loss: 0.3992 - acc: 0.8201 - val_loss: 0.4598 - val_acc: 0.7797


In [31]:
y_pred= classifier.predict(X_test_std_)
y_pred =(y_pred>0.5)
print classification_report(y_pred, y_test_std_)

NameError: name 'Y_test_std_' is not defined